# Membership Inference Attack

## Settings and Imports

In [1]:
# suppress warnings
import warnings

warnings.filterwarnings('ignore')

#autoreload other packages when code changed
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from torch import nn
from torch.utils.data import DataLoader
import torchvision

from opacus.validators import ModuleValidator

import pandas as pd

from tqdm.notebook import tqdm
import random

In [3]:
#Own Code
from privacyflow.configs import path_configs
from privacyflow.datasets import faces_dataset, mi_dataset
from privacyflow.models import face_models, cifar_models, membership_inference_meta_classifier

In [4]:
#Check if GPU is available
if torch.cuda.is_available():
    print("GPU will be used")
    device = torch.device('cuda')
else:
    print("No GPU available")
    device = torch.device('cpu')

GPU will be used


## CIFAR-10 Model - Shadow Modells

In [5]:
cifar10_dataset_train = torchvision.datasets.CIFAR10(root=path_configs.CIFAR_FOLDER_PATH,
                                                     transform=torchvision.transforms.Compose(
                                                         [torchvision.transforms.ToTensor()]
                                                     ),
                                                     train=True,
                                                     download=True)

cifar10_dataset_test = torchvision.datasets.CIFAR10(root=path_configs.CIFAR_FOLDER_PATH,
                                                    transform=torchvision.transforms.Compose(
                                                        [torchvision.transforms.ToTensor()]
                                                    ),
                                                    train=False,
                                                    download=True)

#Combine the datasets for the usage for the shadow models
cifar10_dataset = torch.utils.data.ConcatDataset([cifar10_dataset_train, cifar10_dataset_test])

#Sample the train dataset to only have 10000 items, which matches the number of items in test data
indices = random.sample(range(50000),10000)
cifar10_dataset_train_reduced = torch.utils.data.Subset(cifar10_dataset_train,indices=indices)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
def train_model_no_logs(model: nn.Module,
                        train_dl: torch.utils.data.DataLoader,
                        optimizer: torch.optim,
                        criterion: nn.Module,
                        num_epochs: int = 15):
    model.train()
    model = model.to(device)
    for epoch in range(num_epochs):
        for model_inputs, labels in train_dl:
            model_inputs = model_inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            model_outputs = model(model_inputs)
            loss = criterion(model_outputs, labels)
            loss.backward()
            optimizer.step()

In [ ]:
@torch.no_grad()
def get_mi_data(model: nn.Module,
                datal: torch.utils.data.DataLoader,
                label_included: bool,
                apply_softmax: bool = True):
    model.eval()
    model = model.to(device)
    dfs_batches = []
    for model_inputs, _ in datal:
        model_inputs = model_inputs.to(device)
        preds = model(model_inputs)
        if apply_softmax:
            preds = torch.softmax(preds, dim=-1)
        dfs_batches.append(pd.DataFrame(preds.cpu().detach().numpy()))
    dfs_batches = pd.concat(dfs_batches)
    dfs_batches['target'] = int(label_included)
    return dfs_batches

In [ ]:
def train_shadow_modells_cifar_10_and_get_mi_data(dataset: torch.utils.data.Dataset = cifar10_dataset,
                                                  num_shadow_models: int = 16,
                                                  num_epochs:int=15):
    df_mi_data = []
    for _ in tqdm(range(num_shadow_models), leave=False):
        #Prep Data
        #the original dataset has 50000 train and 10000 test images
        #for the shadow models we use 35000 train images, thus we have 25000 images not included in training
        included_set, excluded_set = torch.utils.data.random_split(dataset, [50000, 10000])
        included_dl = DataLoader(included_set, batch_size=128, num_workers=8, shuffle=True)
        excluded_set = DataLoader(excluded_set, batch_size=128, num_workers=8, shuffle=False)

        #Model Training
        shadow_model = cifar_models.CifarCNNModel(use_log_softmax=False)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(shadow_model.parameters(), lr=0.01)
        train_model_no_logs(model=shadow_model, criterion=criterion, optimizer=optimizer, train_dl=included_dl,num_epochs=num_epochs)
        
        
        #Collect MI_Data
        indices = random.sample(range(50000),10000)
        included_set_reduced = torch.utils.data.Subset(cifar10_dataset_train,indices=indices)
        included_dl_reduced = DataLoader(included_set_reduced , batch_size=128, num_workers=8, shuffle=False)
        df_mi_data.append(get_mi_data(shadow_model, datal=included_dl_reduced, label_included=True, apply_softmax=True))
        df_mi_data.append(get_mi_data(shadow_model, datal=excluded_set, label_included=False, apply_softmax=True))

    return pd.concat(df_mi_data)


In [ ]:
df = train_shadow_modells_cifar_10_and_get_mi_data(num_shadow_models=16)
#df.to_csv(f"{path_configs.MI_DATA_FOLDER}/cifar_shadow_data16.csv", index=False)

## CIFAR-10 Meta Classifier

In [ ]:
mi_ds = mi_dataset.MembershipInferenceDataset(df)
mi_dataloader = DataLoader(mi_ds,
                           batch_size=32,
                           num_workers=4,
                           shuffle=True)

In [ ]:
mi_model = membership_inference_meta_classifier.MIMetaClassifierSmall(input_size=10, output_size=1)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mi_model.parameters(), lr=1e-4)

train_model_no_logs(mi_model,
                    train_dl=mi_dataloader,
                    optimizer=optimizer,
                    criterion=criterion,
                    num_epochs=10)

## CIFAR-10 Membership Inference Attack

In [ ]:
@torch.no_grad()
def get_mi_data_from_attacked_model(
        model: nn.Module,
        dl_included: torch.utils.data.DataLoader,
        dl_excluded: torch.utils.data.DataLoader,
        reverse_log_softmax: bool = True):
    model.eval()
    model = model.to(device)
    df_mi_data = []
    for datal, target in zip([dl_included, dl_excluded], [True, False]):
        for model_inputs, _ in datal:
            model_inputs = model_inputs.to(device)
            model_preds = model(model_inputs)
            if reverse_log_softmax:
                model_preds = torch.exp(model_preds)
            df_batch = pd.DataFrame(model_preds.cpu().detach().numpy())
            df_batch['target'] = int(target)
            df_mi_data.append(df_batch)
    df_mi_data = pd.concat(df_mi_data)
    return df_mi_data


@torch.no_grad()
def eval_mi_attack(
        meta_classifier: nn.Module,
        dl: torch.utils.data.DataLoader) -> float:
    meta_classifier.eval()
    meta_classifier = meta_classifier.to(device)
    num_preds = 0
    num_correct_preds = 0
    for model_inputs, targets in dl:
        model_inputs = model_inputs.to(device)
        targets = targets.to(device)
        preds = meta_classifier(model_inputs)
        num_preds += len(preds)
        num_correct_preds += (preds.round() == targets).sum()
    accuracy_mi = num_correct_preds/num_preds
    print(f"Accuracy MI:{accuracy_mi:.4f}")
    return accuracy_mi

In [ ]:
def mi_attack(
        attacked_model:nn.Module,
        meta_classifier:nn.Module,
        included_dl:torch.utils.data.DataLoader,
        excluded_dl:torch.utils.data.DataLoader,
        reverse_log_softmax:bool=True) -> float:
    #get Preds from attacked model
    df_preds_from_attacked_model = get_mi_data_from_attacked_model(attacked_model,
                                                                   dl_included=included_dl,
                                                                   dl_excluded=excluded_dl,
                                                                   reverse_log_softmax=reverse_log_softmax)
    #turn preds into Dataloader
    mi_attack_ds =mi_dataset.MembershipInferenceDataset(df_preds_from_attacked_model)
    mi_attack_dl = DataLoader(mi_attack_ds,
                           batch_size=64,
                           num_workers=4,
                           shuffle=True)
    #use meta classifier to eval the effektveness of mi attack
    return eval_mi_attack(meta_classifier=meta_classifier,dl=mi_attack_dl)

In [ ]:
attacked_model = cifar_models.CifarCNNModel()
attacked_model.load_state_dict(torch.load(f"{path_configs.MODELS_TRAINED_BASE_PATH}/cifar_10_base.pl"))

acc = mi_attack(attacked_model=attacked_model,
          meta_classifier=mi_model,
          included_dl=DataLoader(cifar10_dataset_train_reduced,batch_size=64,num_workers=4,shuffle=False),
          excluded_dl=DataLoader(cifar10_dataset_test,batch_size=64,num_workers=4,shuffle=False))

In [ ]:
for epsilon in [1,5,10,20,30,50]:
    attacked_model = cifar_models.CifarCNNModel()
    attacked_model = ModuleValidator.fix(attacked_model)
    attacked_model.load_state_dict(torch.load(f"{path_configs.MODELS_TRAINED_BASE_PATH}/cifar_epsilon{epsilon}.pl"))
    
    print(f"Eval Model with Epsilon={epsilon}")
    mi_attack(attacked_model=attacked_model,
          meta_classifier=mi_model,
          included_dl=DataLoader(cifar10_dataset_train_reduced,batch_size=64,num_workers=4,shuffle=False),
          excluded_dl=DataLoader(cifar10_dataset_test,batch_size=64,num_workers=4,shuffle=False))

## ResNet18 Shadow Modells

In [ ]:
train_dataset_celeba = faces_dataset.FacesDataset(label_cols='all', 
                                               mode="train",
                                               transform=torchvision.transforms.Compose(
                                                        [torchvision.transforms.ToTensor()]
                                                    ))
val_dataset_celeba = faces_dataset.FacesDataset(label_cols='all', 
                                             mode="val", 
                                             transform=torchvision.transforms.Compose(
                                                        [torchvision.transforms.ToTensor()]
                                                    ))
test_dataset_celeba = faces_dataset.FacesDataset(label_cols='all', 
                                              mode="test", 
                                              transform=torchvision.transforms.Compose(
                                                        [torchvision.transforms.ToTensor()]
                                                    ))

#Combien Datasets for training of shadow modells
dataset_celeba_combines = torch.utils.data.ConcatDataset([train_dataset_celeba,val_dataset_celeba,test_dataset_celeba])

#Get Data for mi attack on model
indices = random.sample(range(162770),100000)
train_dataset_celeba_reduced = torch.utils.data.Subset(train_dataset_celeba, indices=indices)
indluced_dl_celeba = DataLoader(train_dataset_celeba_reduced,batch_size=64,num_workers=8,shuffle=False)

excluded_ds_celeba = torch.utils.data.ConcatDataset([test_dataset_celeba,val_dataset_celeba])
excluded_dl_celeba = DataLoader(excluded_ds_celeba,batch_size=64,num_workers=8,shuffle=False)

In [ ]:
def train_shadow_modells_resnet18_and_get_mi_data(dataset: torch.utils.data.Dataset,
                                                  num_shadow_models: int = 8):
    df_mi_data = []
    for _ in tqdm(range(num_shadow_models), leave=False):
        #Prep Data
        #the original dataset has 50000 train and 10000 test images
        #for the shadow models we use 35000 train images, thus we have 25000 images not included in training
        included_set, excluded_set = torch.utils.data.random_split(dataset, [150000, 52599])
        included_dl = DataLoader(included_set, batch_size=64, num_workers=8, shuffle=True)
        excluded_set = DataLoader(excluded_set, batch_size=64, num_workers=8, shuffle=False)

        #Model Training
        shadow_model = face_models.get_FaceModelResNet(output_size=40,pretrained=True)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(shadow_model.parameters(), lr=0.01)
        train_model_no_logs(model=shadow_model, 
                            criterion=criterion, 
                            optimizer=optimizer, 
                            train_dl=included_dl,
                            num_epochs=3)
        
        #Collect MI_Data
        indices = random.sample(range(150000),52000)
        included_set_reduced = torch.utils.data.Subset(cifar10_dataset_train,indices=indices)
        included_dl_reduced = DataLoader(included_set_reduced , batch_size=128, num_workers=8, shuffle=False)

        #Collect MI_Data
        df_mi_data.append(get_mi_data(shadow_model, 
                                      datal=included_dl_reduced, 
                                      label_included=True, 
                                      apply_softmax=False))
        df_mi_data.append(get_mi_data(shadow_model, 
                                      datal=excluded_set, 
                                      label_included=False, 
                                      apply_softmax=False))

    return pd.concat(df_mi_data)

In [ ]:
df = train_shadow_modells_resnet18_and_get_mi_data(dataset=dataset_celeba_combines, num_shadow_models=8)
df.to_csv(f"{path_configs.MI_DATA_FOLDER}/celeba_resnet18_shadow_data8.csv", index=False)

In [ ]:
df = pd.read_csv(f"{path_configs.MI_DATA_FOLDER}/celeba_resnet18_shadow_data8.csv")

In [ ]:
df= df.groupby('target').sample(n=740000)

In [ ]:
#Train Meta Classifier
mi_ds = mi_dataset.MembershipInferenceDataset(df)
mi_dataloader = DataLoader(mi_ds,
                           batch_size=16,
                           num_workers=4,
                           shuffle=True)

mi_model = membership_inference_meta_classifier.MIMetaClassifierMedium(input_size=40, output_size=1)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mi_model.parameters(), lr=1e-8)
train_model_no_logs(mi_model,
                    train_dl=mi_dataloader,
                    optimizer=optimizer,
                    criterion=criterion,
                    num_epochs=15)

In [ ]:
# Attack Base Model
attacked_model = face_models.get_FaceModelResNet(output_size=40,pretrained=False)
attacked_model.load_state_dict(torch.load(f"{path_configs.MODELS_TRAINED_BASE_PATH}/face_base_model.pl"))
mi_attack(attacked_model=attacked_model,
          meta_classifier=mi_model,
          included_dl=indluced_dl_celeba,
          excluded_dl=excluded_dl_celeba)

In [ ]:
# Attack DPSGD Models
for epsilon in [1,5,10]:
    #Load DPSGD Model
    attacked_model=face_models.get_FaceModelResNet(output_size=40,pretrained=False)
    attacked_model = ModuleValidator.fix(attacked_model)
    attacked_model.load_state_dict(torch.load(f"{path_configs.MODELS_TRAINED_BASE_PATH}/cnn_pretrained_epsilon{epsilon}_epochs3_clipp1e-05_batch256_ohneAA.pl"))
    #Eval MI
    print(f"Eval Model with Epsilon={epsilon}")
    mi_attack(attacked_model=attacked_model,
          meta_classifier=mi_model,
          included_dl=indluced_dl_celeba,    
          excluded_dl=excluded_dl_celeba)